In [1]:
## Script to analyze Turbo Typing data

Notes on where I'm leaving off:   
- 9/22/24 Just wrote code that removes the extra spaces before characters from the imported data without deleting actual spaces. Next I want to use the collapsed sentences with no extra s
- spaces to use the diff function and compare them.  
- 9/23/2024 Basically same goal for next time. I had to make a new column that replaces the multi-character key names with special characters for the keyData from the participants, which was completed today.
- 9/24/2024 Isolated correct keypress times. Next time, calculate ttk for correct presses, mean ttk, and the rest of the assessment variables. Also ask Patrick the best way to organize and write code. 

In [2]:
import os
import numpy as np
import pandas as pd
import math
import glob
import editdistance
import diff_match_patch as dmp_module
dmp = dmp_module.diff_match_patch()
import copy

In [3]:
# The imported data from Turbo Typing stores listed data as a long string. 
# This splits it up into actual lists.
def str_to_list(dataframe, columns):
    for col in columns:
        data_str = dataframe[col]
        if type(data_str.iloc[0]) == list:
            pass
        elif type(data_str.iloc[0]) == str:
            if data_str.iloc[0][0].isdigit() == True:
                data_list = data_str.apply(lambda trial: 
                                           [int(char) for char in trial.split(',')])
                dataframe[col] = data_list
            elif data_str.iloc[0][0].isdigit() == False:
                data_list = [x.split(',') for x in data_str]
                dataframe[col] = data_list
        else:
            print('Column must have data in string form.')

# Removes extra spaces before characters in keypress data.
def no_extra_spaces(dataframe, columns):
    for col in columns:
        no_space = dataframe[col].apply(lambda trial: 
                                        [char.replace(" ","") if char.isspace() 
                                         != True else ' ' for char in trial])
        dataframe[col] = no_space

# Changes multi-chracter keynames (ie. LeftShift and Backspace) to special
# characters.
def multi_to_special(dataframe, column):
    data = dataframe[column]
    edit_data = []
    for trial in data:
        edit_trial = copy.deepcopy(trial) # Pandas can't deep copy lists in dataframes, so this is a work around.
        for index, key in enumerate(edit_trial):
            if len(key) == 1:
                pass
            if len(key) > 1:
                if key in ['LeftShift', 'RightShift']:
                    edit_trial[index] = '#'
                elif key == 'Backspace':
                    edit_trial[index] = '-'
                elif key == 'Return':
                    edit_trial[index] = '^'
                else:
                    edit_trial[index] = '~' # '~' replaces any other keys longer than one character.
        edit_data.append(edit_trial)
    return edit_data

# Calculates time to keypress (TTK) for all presses in one trial.
# TTK is the reaction time for the first press and the interkey 
# interval for any subsequent presses.
def ttk(row):
    presses = pd.DataFrame(row.loc['timeData'])
    ttk_df = presses.sub(presses.shift(fill_value=0))
    ttk_list = ttk_df.values.flatten().tolist()
    return ttk_list

# Creates a list of keys that would have been pressed if the sentence was typed
# with no errors (ie. no backspacing). Uses single, special characters to 
# represent multi-character key names (ie. '^' for 'Return').
def sentence_to_keydata(string):
    curr_keyData = []
    for index, data in enumerate(string):
        if data.isupper() == True:
            curr_keyData.append('#') # '#' Replaces 'Right/LeftShift'
            curr_keyData.append(data)
        elif data.isupper() == False and data.isalpha() or data.isspace() == True:
            curr_keyData.append(data)
        elif data == '?':
            curr_keyData.append('#')
            curr_keyData.append(data)
            curr_keyData.append('^')
        elif data == '.':
            curr_keyData.append(data)
            curr_keyData.append('^') # '^' Replaces 'Return'
    return curr_keyData

# # Splits up keyDataConverted data from a string to a list of pressed keys.
# def split_str(row):
#     keys_str = row.loc['keyDataConverted']
#     keys_list = keys_str.split(',')
#     return keys_list

In [4]:
# Setting save directory
save_dir = os.getcwd()
save_dir

'/Users/rubi/Desktop/Github/typingexp/typing_task_analysis'

In [5]:
# Getting subject data folders from server.
server = r'/Volumes/greenhouse/typingtask_data/subject_data'
os.chdir(server)
folders = os.listdir()

# Looping through subject folders, getting appropriate paths to data, 
# and making sID list.
sub_folders = list(filter(lambda x: x.startswith('s', 0, 1), folders))
all_turbo = pd.DataFrame()
ID_list = []
for sub in sub_folders:
    sub_folder = r'/Volumes/greenhouse/typingtask_data/'\
                 'subject_data/%s/turbotyping_data' % sub
    os.chdir(sub_folder)
    sID = sub.split('_', 1)[0]
    turbo = pd.read_csv(glob.glob('*_datafile.tsv')[0], sep='\t')
    all_turbo = pd.concat([all_turbo, turbo])
    ID_list.append(sID)

all_turbo = all_turbo.reset_index()

In [6]:
# Transforms listed data stored in long strings to actual lists.
str_to_list(all_turbo, ['timeData', 
                        'keyData', 
                        'keyDataConverted', 
                        'timeDataUp',
                        'keyDataUp'])

# Removes extra spaces before characters in keypress data columns.
no_extra_spaces(all_turbo, ['keyData', 'keyDataConverted', 'keyDataUp'])

# Replaces multi-character key names with single, special characters from key
# press data typed by participants. 
no_multichar_keys = multi_to_special(all_turbo, 'keyDataConverted')
all_turbo.insert(11, 'keyDataSpecialChar', no_multichar_keys)

# Creates column that represents the correct key press sequence if there were
# no errors (uses special characters for mtuli-character key names).
current_keyData = [sentence_to_keydata(x) for x in all_turbo['currentSentence']]
all_turbo.insert(8, 'keyDataCurrent', current_keyData)

In [7]:
# Calculates edit distance of each trial and stores to an additional column.
edit_dist = all_turbo.apply(lambda row: 
                            editdistance.eval(row.loc['currentSentence'], 
                                              row.loc['typedSentence']), 
                            axis=1)
all_turbo['editDistance'] = edit_dist

# Calculates total number of correct keypressess and stores to an 
# additional column.
### EDIT THIS THE NUMBER OF CORRECT KEYPRESSES IS DIFFERENT THAN THE SENTENCE ###
num_corr = all_turbo.apply(lambda row:
                           (len(row.loc['currentSentence']) 
                            - row.loc['editDistance']),
                           axis=1)
all_turbo['numCorr'] = num_corr

# Calculates TTK for every character in each trial and stores to 
# an additional column.
ttks = all_turbo.apply(ttk, axis=1)
all_turbo['ttk'] = ttks

# Calculates mean TTK for each trial and stores to an additional column.
mean_ttk = [np.mean(x) for x in all_turbo['ttk']]
all_turbo['ttkMean'] = mean_ttk

# Calculates sum of all TTKs for each trial and stores to an additional column.
sum_ttk = [np.sum(x) for x in all_turbo['ttk']]
all_turbo['ttkSum'] = sum_ttk
all_turbo

,index,participantID,trialNumber,currentSentence,typedSentence,trialStart,wpm,keyDataCurrent,accuracy,timeData,keyData,keyDataSpecialChar,keyDataConverted,timeDataUp,keyDataUp,editDistance,numCorr,ttk,ttkMean,ttkSum
0,0,262,-2,Looking for some gift hints from you.,Looking for some gift hints from you.,1/23/2024 9:17:58 AM,86,"[#, L, o, o, k, i, n, g, , f, o, r, , s, o, ...",100,"[21454, 21906, 22156, 22284, 22371, 22611, 226...","[LeftShift, L, O, O, K, I, N, G, Space, F, O, ...","[#, L, o, o, k, i, n, g, , f, o, r, , s, o, ...","[LeftShift, L, o, o, k, i, n, g, , f, o, r, ...","[22043, 22051, 22220, 22411, 22500, 22691, 227...","[L, LeftShift, O, O, K, I, N, G, Space, F, O, ...",0,37,"[21454, 452, 250, 128, 87, 240, 17, 120, 79, 1...",651.195122,26699
1,1,262,-1,I can't tell you when they'll be back.,I can't tell you when they'll be back.,1/23/2024 9:18:28 AM,63,"[#, I, , c, a, n, t, , t, e, l, l, , y, o, ...",100,"[22442, 22633, 22968, 23041, 23498, 23809, 239...","[LeftShift, I, C, A, Backspace, Backspace, Spa...","[#, I, c, a, -, -, , c, a, n, ', t, , t, e, ...","[LeftShift, I, c, a, Backspace, Backspace, , ...","[22769, 22825, 23081, 23131, 23562, 23890, 240...","[I, LeftShift, C, A, Backspace, Backspace, Spa...",0,38,"[22442, 191, 335, 73, 457, 311, 136, 104, 57, ...",665.681818,29290
2,2,262,0,Our forces are ready to come immediately.,Our forces are ready to come immedaitely.,1/23/2024 9:19:04 AM,80,"[#, O, u, r, , f, o, r, c, e, s, , a, r, e, ...",95,"[927, 1118, 1311, 1494, 1631, 1718, 1854, 1903...","[LeftShift, LeftShift, O, U, R, Space, F, O, R...","[#, #, O, u, r, , f, o, r, c, e, s, , a, r, ...","[LeftShift, LeftShift, O, u, r, , f, o, r, c,...","[1071, 1470, 1478, 1655, 1758, 1846, 1926, 199...","[LeftShift, LeftShift, O, U, R, Space, F, O, R...",2,39,"[927, 191, 193, 183, 137, 87, 136, 49, 103, 19...",147.954545,6510
3,3,262,1,A bad thing has been turned into a good thing.,A bad thing has been turned into a good thing.,1/23/2024 9:19:12 AM,108,"[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...",100,"[586, 857, 970, 1146, 1281, 1346, 1458, 1554, ...","[LeftShift, A, Space, B, A, D, Space, T, H, I,...","[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...","[LeftShift, A, , b, a, d, , t, h, i, n, g, ...","[962, 994, 1066, 1211, 1402, 1449, 1601, 1681,...","[LeftShift, A, Space, B, A, D, Space, T, H, I,...",0,46,"[586, 271, 113, 176, 135, 65, 112, 96, 80, 72,...",110.541667,5306
4,4,262,2,Do you want to eat lunch somewhere before?,Do you want to eat lunch somewhere before?,1/23/2024 9:19:19 AM,98,"[#, D, o, , y, o, u, , w, a, n, t, , t, o, ...",100,"[530, 730, 858, 962, 1065, 1130, 1194, 1266, 1...","[LeftShift, D, O, Space, Y, O, U, Space, W, A,...","[#, D, o, , y, o, u, , w, a, n, t, , t, o, ...","[LeftShift, D, o, , y, o, u, , w, a, n, t, ...","[802, 841, 946, 1114, 1186, 1234, 1313, 1394, ...","[LeftShift, D, O, Space, Y, O, U, Space, W, A,...",0,42,"[530, 200, 128, 104, 103, 65, 64, 72, 95, 65, ...",114.533333,5154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,12,s309_09232024,10,A bad thing has been turned into a good thing.,A bad thing has been turned into a good thing.,9/23/2024 2:35:29 PM,45,"[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...",100,"[945, 1169, 1361, 1576, 1769, 2032, 2489, 2664...","[LeftShift, A, Space, B, A, D, Space, T, H, I,...","[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...","[LeftShift, A, , b, a, d, , t, h, i, n, g, ...","[1257, 1304, 1489, 1680, 1912, 2137, 2609, 276...","[LeftShift, A, Space, B, A, D, Space, T, H, I,...",0,46,"[945, 224, 192, 215, 193, 263, 457, 175, 97, 2...",269.375000,12930
370,13,s309_09232024,11,We talked quietly while everybody gathered.,We talked quietly while everybody gathered.,9/23/2024 2:35:44 PM,32,"[#, W, e, , t, a, l, k, e, d, , q, u, i, e, ...",100,"[736, 967, 1304, 1792, 2504, 2904, 3048, 3256,...","[LeftShift, W, E, T, Backspace, Space, T, A, L...","[#, W, e, t, -, , t, a, l, k, e, d, , q, u, ...","[LeftShift, W, e, 

In [8]:
# Compares the correct sequence of keypresses with what was typed during the 
# trial and returns the difference between the two.
string_diff = all_turbo.apply(lambda row: 
                            dmp.diff_main(''.join(row.loc['keyDataSpecialChar']), 
                                          ''.join(row.loc['keyDataCurrent'])),
                            axis=1)

# Separates the returned differences into individual characters.
diffs_bychar = []
for trial in string_diff:
    diff_bychar_trial = []
    for diff in trial:
        if len(diff[1]) == 1:
            diff_bychar_trial.append([diff[0], diff[1]])
        elif len(diff[1]) > 1:
            for char in diff[1]:
                diff_bychar_trial.append([diff[0], char])
    diffs_bychar.append(diff_bychar_trial)

# Differences for errors where participants swap two letters are solved as: 
# letter 2 subtraction, letter 1 retention, letter 2 addition. Because  
# both letters 1 and 2 need to be identified as incorrect keypresses, this code 
# changes the difference to show: letter 2 subtraction, letter 1 subtraction.
diffs_bychar_fixswap = []
for trial in diffs_bychar:
    chardiffs_trial = []
    char_toskip = 0
    for index, char in enumerate(trial):
        if char_toskip != 0:
            char_toskip = char_toskip - 1
            continue
        if char[0] == 0:
            if index == (len(trial) - 1) or index == 0:
                chardiffs_trial.append(char)
            else:
                if trial[index - 1][0] == -1 and trial[index + 1][0] == 1:
                    if trial[index - 1][1] == trial[index + 1][1]:
                        chardiffs_trial.append([-1, char[1]])
                        char_toskip = 1
                else:
                    chardiffs_trial.append(char)
        else:
            chardiffs_trial.append(char)
    diffs_bychar_fixswap.append(chardiffs_trial)

In [9]:
corr_keys = []
corr_indices = []
for trial in diffs_bychar_fixswap:
    corr_keys_trial = []
    corr_indices_trial = []
    for index, char in enumerate(trial):
        if char[0] == 0:
            corr_keys_trial.append(char[1])
            corr_indices_trial.append(index)
        else:
            pass
    corr_keys.append(corr_keys_trial)
    corr_indices.append(corr_indices_trial)
    
all_turbo.insert(12, 'keyDataCorrect', corr_keys)
all_turbo.insert(13, 'keyIndicesCorrect', corr_indices)
all_turbo

,index,participantID,trialNumber,currentSentence,typedSentence,trialStart,wpm,keyDataCurrent,accuracy,timeData,...,keyDataCorrect,keyIndicesCorrect,keyDataConverted,timeDataUp,keyDataUp,editDistance,numCorr,ttk,ttkMean,ttkSum
0,0,262,-2,Looking for some gift hints from you.,Looking for some gift hints from you.,1/23/2024 9:17:58 AM,86,"[#, L, o, o, k, i, n, g, , f, o, r, , s, o, ...",100,"[21454, 21906, 22156, 22284, 22371, 22611, 226...",...,"[#, L, o, o, k, i, n, g, , f, o, r, , s, o, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[LeftShift, L, o, o, k, i, n, g, , f, o, r, ...","[22043, 22051, 22220, 22411, 22500, 22691, 227...","[L, LeftShift, O, O, K, I, N, G, Space, F, O, ...",0,37,"[21454, 452, 250, 128, 87, 240, 17, 120, 79, 1...",651.195122,26699
1,1,262,-1,I can't tell you when they'll be back.,I can't tell you when they'll be back.,1/23/2024 9:18:28 AM,63,"[#, I, , c, a, n, t, , t, e, l, l, , y, o, ...",100,"[22442, 22633, 22968, 23041, 23498, 23809, 239...",...,"[#, I, , c, a, n, t, , t, e, l, l, , y, o, ...","[0, 1, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17,...","[LeftShift, I, c, a, Backspace, Backspace, , ...","[22769, 22825, 23081, 23131, 23562, 23890, 240...","[I, LeftShift, C, A, Backspace, Backspace, Spa...",0,38,"[22442, 191, 335, 73, 457, 311, 136, 104, 57, ...",665.681818,29290
2,2,262,0,Our forces are ready to come immediately.,Our forces are ready to come immedaitely.,1/23/2024 9:19:04 AM,80,"[#, O, u, r, , f, o, r, c, e, s, , a, r, e, ...",95,"[927, 1118, 1311, 1494, 1631, 1718, 1854, 1903...",...,"[#, O, u, r, , f, o, r, c, e, s, , a, r, e, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[LeftShift, LeftShift, O, u, r, , f, o, r, c,...","[1071, 1470, 1478, 1655, 1758, 1846, 1926, 199...","[LeftShift, LeftShift, O, U, R, Space, F, O, R...",2,39,"[927, 191, 193, 183, 137, 87, 136, 49, 103, 19...",147.954545,6510
3,3,262,1,A bad thing has been turned into a good thing.,A bad thing has been turned into a good thing.,1/23/2024 9:19:12 AM,108,"[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...",100,"[586, 857, 970, 1146, 1281, 1346, 1458, 1554, ...",...,"[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[LeftShift, A, , b, a, d, , t, h, i, n, g, ...","[962, 994, 1066, 1211, 1402, 1449, 1601, 1681,...","[LeftShift, A, Space, B, A, D, Space, T, H, I,...",0,46,"[586, 271, 113, 176, 135, 65, 112, 96, 80, 72,...",110.541667,5306
4,4,262,2,Do you want to eat lunch somewhere before?,Do you want to eat lunch somewhere before?,1/23/2024 9:19:19 AM,98,"[#, D, o, , y, o, u, , w, a, n, t, , t, o, ...",100,"[530, 730, 858, 962, 1065, 1130, 1194, 1266, 1...",...,"[#, D, o, , y, o, u, , w, a, n, t, , t, o, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[LeftShift, D, o, , y, o, u, , w, a, n, t, ...","[802, 841, 946, 1114, 1186, 1234, 1313, 1394, ...","[LeftShift, D, O, Space, Y, O, U, Space, W, A,...",0,42,"[530, 200, 128, 104, 103, 65, 64, 72, 95, 65, ...",114.533333,5154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,12,s309_09232024,10,A bad thing has been turned into a good thing.,A bad thing has been turned into a good thing.,9/23/2024 2:35:29 PM,45,"[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...",100,"[945, 1169, 1361, 1576, 1769, 2032, 2489, 2664...",...,"[#, A, , b, a, d, , t, h, i, n, g, , h, a, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[LeftShift, A, , b, a, d, , t, h, i, n, g, ...","[1257, 1304, 1489, 1680, 1912, 2137, 2609, 276...","[LeftShift, A, Space, B, A, D, Space, T, H, I,...",0,46,"[945, 224, 192, 215, 193, 263, 457, 175, 97, 2...",269.375000,12930
370,13,s309_09232024,11,We talked quietly while everybody gathered.,We talked quietly while everybody gathered.,9/23/2024 2:35:44 PM,32,"[#, W, e, , t, a, l, k, e, d, , q, u, i, e, ...",100,"[736, 967, 1304, 1792, 2504, 2904, 3048, 3256,...",...,"[#, W, e, , t, a, l, k, e, d, , q, u, i, e, ...","[0, 1, 2, 5, 6, 7, 8, 9, 10, 

In [13]:
# Isolates keypress times for correct keypresses.
def corr_time(trial):
    times = trial.loc['timeData']
    indices = trial.loc['keyIndicesCorrect']
    matched_indices =[]
    for index, time in enumerate(times):
        if index in indices:
            matched_indices.append(time)
    return matched_indices

# Isolates keypress times for correct keypresses and adds to a new column.
corr_times = all_turbo.apply(corr_time, axis=1)
all_turbo.insert(10, 'timeDataCorrect', corr_times)

0      [21454, 21906, 22156, 22284, 22371, 22611, 226...
1      [22442, 22633, 23945, 24049, 24106, 24241, 246...
2      [1118, 1311, 1494, 1631, 1718, 1854, 1903, 200...
3      [586, 857, 970, 1146, 1281, 1346, 1458, 1554, ...
4      [530, 730, 858, 962, 1065, 1130, 1194, 1266, 1...
                             ...                        
369    [945, 1169, 1361, 1576, 1769, 2032, 2489, 2664...
370    [736, 967, 1304, 2904, 3048, 3256, 3376, 3528,...
371    [511, 1079, 1271, 1440, 1887, 2096, 2432, 2536...
372    [962, 1282, 1521, 1674, 1866, 2026, 2265, 2602...
373    [697, 808, 983, 1080, 1232, 1464, 1608, 1800, ...
Name: timeDataCorrect, Length: 374, dtype: object

In [ ]:
# Calculates rate correct score (RCS) for each trail and stores in column.
rcs = all_turbo.apply(lambda row: 
                      row.loc['numCorr']/row.loc['ttkSum'],
                      axis=1)
all_turbo['rcs'] = rcs

# Calculates linear integrated speed-ccuracy score (LISAS) and stores in column.
# lisas = 

In [ ]:



# for trial in data_str:
#     data_list = [int(x) for x in trial.split(',')]
# data_list
# for trial in data_list:
#     data_int = 
# data_list[0]
# data_int = [print(x) for x in data_list]
# data_int

# edit_dist = all_turbo.apply(lambda row: 
#                             editdistance.eval(row.loc['currentSentence'], 
#                             row.loc['typedSentence']), 
#                             axis=1)